In [ ]:
import cv2
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

In [ ]:
def load_images_from_folder(folder):
    images = []
    labels = []
    for subfolder in os.listdir(folder):
        category = subfolder
        for filename in os.listdir(os.path.join(folder, subfolder)):
            img = cv2.imread(os.path.join(folder, subfolder, filename))
            if img is not None:
                img = cv2.resize(img,(64,64))
                img = img.flatten()
                images.append(img)
                labels.append(category)
    return images, labels

train_images, train_labels = load_images_from_folder("images/train")
test_images, test_labels = load_images_from_folder("images/test")

train_images = np.array(train_images)
test_images = np.array(test_images)

le = LabelEncoder()
train_labels = le.fit_transform(train_labels)
test_labels = le.transform(test_labels)

pca = PCA(n_components=50)

train_images = pca.fit_transform(train_images)
test_images = pca.transform(test_images)

model = LogisticRegression(random_state=0).fit(train_images, train_labels)

predict_labels = model.predict(test_images)

print("Accuracy: %.2f%%" % (accuracy_score(test_labels, predict_labels)*100))

In [ ]:
def load_images_from_folder(folder):
    images = []
    labels = []
    model = VGG16(weights='imagenet', include_top=False)
    for subfolder in os.listdir(folder):
        category = subfolder
        for filename in os.listdir(os.path.join(folder, subfolder)):
            img = image.load_img(os.path.join(folder, subfolder, filename), target_size=(224, 224))
            img = image.img_to_array(img)
            img = np.expand_dims(img, axis=0)
            img = preprocess_input(img)

            features = model.predict(img)
            features = features.flatten()
            images.append(features)
            labels.append(category)
    return images, labels

train_images, train_labels = load_images_from_folder("images/train")
test_images, test_labels = load_images_from_folder("images/test")

train_images = np.array(train_images)
test_images = np.array(test_images)

le = LabelEncoder()
train_labels = le.fit_transform(train_labels)
test_labels = le.transform(test_labels)

model = LogisticRegression(random_state=0).fit(train_images, train_labels)
predict_labels = model.predict(test_images)

print("Accuracy: %.2f%%" % (accuracy_score(test_labels, predict_labels)*100))